In [7]:
from qiskit import IBMQ
# from qiskit.providers.aer.pulse.hamiltonian_model import HamiltonianModel
from qiskit.providers.aer.pulse.system_models.hamiltonian_model import HamiltonianModel
from qiskit.test.mock import FakeAlmaden, FakeArmonk, FakeAthens
from qutip import sigmax, sigmay, Qobj, sigmaz
from src.prefactor_parsing import prefactor_parser

In [8]:
input_backend = FakeAlmaden()
subsystem_list=[0,1]
two_level=True

In [9]:
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
real_backend = provider.get_backend('ibmq_athens')
hamiltonian_backend = real_backend.configuration().hamiltonian


/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2020-08-24 10:03:23,923: Credentials are already in use. The existing account in the session will be replaced.


In [10]:
print(hamiltonian_backend['h_latex'])
# hamiltonian_dict = HamiltonianModel.from_dict(hamiltonian_backend)

\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{4}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{2,3}(\sigma_{2}^{+}\sigma_{3}^{-}+\sigma_{2}^{-}\sigma_{3}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{3}^{(2,1)}(t)+U_{4}^{(2,3)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{6}^{(3,4)}(t)+U_{5}^{(3,2)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{7}^{(4,3)}(t))\sigma_{4}^{X} \\ \end{align}


\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{4}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{2,3}(\sigma_{2}^{+}\sigma_{3}^{-}+\sigma_{2}^{-}\sigma_{3}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{3}^{(2,1)}(t)+U_{4}^{(2,3)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{6}^{(3,4)}(t)+U_{5}^{(3,2)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{7}^{(4,3)}(t))\sigma_{4}^{X} \\ \end{align}

In [11]:
# 11AM started from_dict

In [12]:


hamiltonian_dict = HamiltonianModel.from_dict(hamiltonian_backend, subsystem_list=subsystem_list)

In [26]:
print(hamiltonian_dict._system[0][0])

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         1.41421356 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  1.41421356 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         2.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]]


In [13]:
hamiltonian = {'H_c': {}, 'iH_c': {}, 'H_d': 0}

In [14]:
for i, control_field in enumerate(hamiltonian_dict._system):
    matrix = (control_field[0])
    prefactor, control = prefactor_parser(control_field[1], hamiltonian_dict._variables)
    if prefactor == 0:
        continue
    if control:
        if two_level:
            hamiltonian['H_c'][control] = sigmax() * prefactor
            hamiltonian['iH_c'][control] = sigmay() * prefactor
        else:
            hamiltonian['H_c'][control] = (matrix * prefactor)#.full()
            hamiltonian['iH_c'][control] = Qobj((complex(0, 1) * matrix * prefactor).full())
    elif two_level:
        hamiltonian['H_d'] = sigmaz() * prefactor
    elif hamiltonian['H_d'] == 0:
        hamiltonian['H_d'] = matrix * prefactor
    else:
        print(matrix, prefactor)
        hamiltonian['H_d'] += matrix * prefactor
hamiltonian['H_d'] = Qobj(hamiltonian['H_d'].full())